# Decorator Middleware

This notebook demonstrates how to use @agent_middleware and @function_middleware decorators.

## Key Concepts

- **No Type Annotations Needed**: Use decorators without parameter types
- **Explicit Middleware Type**: Decorator marks the middleware type
- **Framework Handles Context**: Framework automatically provides correct context types

## Prerequisites

- Azure AI Foundry project endpoint configured in `.env`
- `agent-framework` package installed
- Azure CLI authentication

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"Project Endpoint: {project_endpoint}")
print(f"Model Deployment: {model_deployment_name}")

Project Endpoint: https://kd-foundry-project-resource.services.ai.azure.com/api/projects/kd-foundry-project
Model Deployment: gpt-4o


In [7]:
import datetime

from agent_framework import (
    agent_middleware,
    function_middleware,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

## Define Tool Function

In [8]:
def get_current_time() -> str:
    """Get the current time."""
    return f"Current time is {datetime.datetime.now().strftime('%H:%M:%S')}"

## Decorator-Based Middleware

This sample demonstrates how to use @agent_middleware and @function_middleware decorators
to explicitly mark middleware functions without requiring type annotations.

The framework supports the following middleware detection scenarios:

1. Both decorator and parameter type specified:
   - Validates that they match (e.g., @agent_middleware with AgentRunContext)
   - Throws exception if they don't match for safety

2. Only decorator specified:
   - Relies on decorator to determine middleware type
   - No type annotations needed - framework handles context types automatically

3. Only parameter type specified:
   - Uses type annotations (AgentRunContext, FunctionInvocationContext) for detection

4. Neither decorator nor parameter type specified:
   - Throws exception requiring either decorator or type annotation
   - Prevents ambiguous middleware that can't be properly classified

Key benefits of decorator approach:
- No type annotations needed (simpler syntax)
- Explicit middleware type declaration
- Clear intent in code
- Prevents type mismatches

In [9]:
@agent_middleware  # Decorator marks this as agent middleware - no type annotations needed
async def simple_agent_middleware(context, next):  # type: ignore - parameters intentionally untyped to demonstrate decorator functionality
    """Agent middleware that runs before and after agent execution."""
    print("[Agent Middleware] Before agent execution")
    await next(context)
    print("[Agent Middleware] After agent execution")


@function_middleware  # Decorator marks this as function middleware - no type annotations needed
async def simple_function_middleware(context, next):  # type: ignore - parameters intentionally untyped to demonstrate decorator functionality
    """Function middleware that runs before and after function calls."""
    print(f"[Function Middleware] Before calling: {context.function.name}")  # type: ignore
    await next(context)
    print(f"[Function Middleware] After calling: {context.function.name}")  # type: ignore

## Example: Using Decorator Middleware

In [10]:
async def main():
    """Example demonstrating decorator-based middleware."""
    print("=== Decorator Middleware Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="TimeAgent",
            instructions="You are a helpful time assistant. Call get_current_time when asked about time.",
            tools=get_current_time,
            middleware=[simple_agent_middleware, simple_function_middleware],
        ) as agent,
    ):
        query = "What time is it?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result.text if result.text else 'No response'}")

await main()

=== Decorator Middleware Example ===
User: What time is it?
[Agent Middleware] Before agent execution


[2025-10-25 01:23:38 - c:\Users\kapildhanger\OneDrive - Microsoft\Microsoft_Kapil\AzureCustomers\Everest\NYC_Workshop\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[Function Middleware] Before calling: get_current_time
[Function Middleware] After calling: get_current_time
[Agent Middleware] After agent execution
Agent: The current time is 01:23:38.
[Agent Middleware] After agent execution
Agent: The current time is 01:23:38.


## Key Takeaways

- Decorators (@agent_middleware, @function_middleware) explicitly mark middleware type
- No type annotations needed when using decorators
- Framework automatically provides correct context types
- Prevents type mismatches and ambiguous middleware
- Simpler and cleaner syntax

## Next Steps

- **Chat middleware** (notebook 5) for message-level interception
- **Exception handling** (notebook 6) for error management